# Read the data

In [3]:
import os
import pandas as pd

from fastapi import FastAPI
from typing import List
from models import CountryData

In [5]:
filepath = "../data/TPI ASCOR data - 13012025/ASCOR_assessments_results.xlsx" # Specify the correct path to the file
df_assessments = pd.read_excel(filepath)

# Convert the date columns to datetime type so we can filter by year later
df_assessments['Assessment date'] = pd.to_datetime(df_assessments['Assessment date'])
df_assessments['Publication date'] = pd.to_datetime(df_assessments['Publication date'])

/var/folders/kv/l9vcg1gs7xb1tfhh37wfc4_w0000gn/T/ipykernel_31605/3951065428.py:5: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_assessments['Assessment date'] = pd.to_datetime(df_assessments['Assessment date'])


# Define the function

In [20]:
def get_country_data(country: str, assessment_year: int):
    data = df_assessments[(df_assessments["Country"] == country) & (df_assessments["Assessment date"].dt.year == assessment_year)]

    #filter for the area columns
    area_columns = [col for col in data.columns if col.startswith("area")]
    data = data[area_columns]
    data['country'] = country
    data['assessment_year'] = assessment_year

    #JSON does not allow for NaN or NULL
    data = data.fillna("")

    #rename columns
    remap_area_column_names = {
        col: col.replace('area ', '').replace(".", "_") for col in area_columns
    }

    data = data.rename(columns=remap_area_column_names)

    #Grab just the first row and return it as a dictionary.
    #Even though there should only be one row anyway, we specify it because we want to convert to dictionary
    output_dict = data.iloc[0].to_dict()

    output = CountryData(**output_dict)

    return output

# Test how the function behaves

In [21]:
get_country_data("Italy", 2024)

CountryData(country='Italy', assessment_year=2024, EP_1='Partial', EP_2='Partial', EP_3='Partial', CP_1='No', CP_2='Partial', CP_3='Partial', CP_4='Partial', CP_5='Partial', CP_6='Partial', CF_1='No', CF_2='Exempt', CF_3='Partial', CF_4='')

# Testing out Pydantic Models

In [ ]:
from pydantic import BaseModel